<a href="https://colab.research.google.com/github/Carapet07/Data-Science-projects/blob/main/First_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##My first NLP project

In [2]:
import os, pathlib, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  3156k      0  0:00:26  0:00:26 --:--:-- 4390k
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [4]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [5]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
for text_batch, label_batch in train_ds.take(1):  # Take one batch
    for i in range(5):  # Show first 5 examples
        print(f"Text: {text_batch.numpy()[i].decode('utf-8')}")
        print(f"Label: {label_batch.numpy()[i]}")
        print("=" * 50)


Text: I was babysitting a family of three small children for a night and their mother gave me this to show for them having just grabbed it at Wal-Mart earlier in the week. All three children actually got physically ill while watching it. I'm pretty sure it was the pizza they ate, or something they all had picked up from school, but really it could have been this film. Absolutely disgusting. How any one can produce this caliber of trash is beyond me. Fortunately, I turned off the film when I noticed the children were not responding and acting strangely. For any parents out there, I strongly advise you to refrain from letting young children view this movie.
Label: 0
Text: We do not come across movies on brother-sister relationship in Indian cinema, or any other language or medium. This relationship has several aspects which have not been exploited in movies or novels. Typically, a sister is depicted as a pile-on who can be used for ransom in the climax. This movie treats the subject in a

In [7]:
max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)

In [8]:
for inputs, targets in train_ds:
  print("input shape: ", inputs.shape)
  print("input dtype: ", inputs.dtype)
  print("targets shape: ", targets.shape)
  print("targets dtype: ", targets.dtype)
  print('inputs[0]: ', inputs[0])
  print('targets[0]: ', targets[0])
  break

input shape:  (32,)
input dtype:  <dtype: 'string'>
targets shape:  (32,)
targets dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b'Thunderbirds (2004) <br /><br />Director: Jonathan Frakes <br /><br />Starring: Bill Paxton, Ben Kingsley, Brady Corbet <br /><br />5\xc2\x854\xc2\x853\xc2\x852\xc2\x851! Thunderbirds are GO! <br /><br />And so began Thunderbirds, a childhood favorite of mine. When I heard that they were going to make a Thunderbirds movie, I was ecstatic. I couldn\'t wait to see Thunderbird 2 roar in to save people, while Thunderbird 4 would dive deep into the\xc2\x85you get the idea. I just couldn\'t wait. Then came August 2004, when the movie was finally released. Critics panned it, but I still wanted to go. After all, as long as the heart was in the same place, that was all that mattered to me. So I sat down in the theater, the only teenager in a crowd of 50\xc2\x85everyone else was over thirty and under ten. Quite possibly the most awkward theater experience that I have

This dataset yields inputs that are tf.string tensors and targets that are int32 tensors encoding the value "0" or "1"


All set, now let's try learnign something from this data


In [11]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=128, input_length=max_length)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 128)           │       2,560,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,601,281 (9.92 MB)

 Trainable params: 2,601,281 (9.92 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                      save_best_only=True)
    ]

history = model.fit(int_train_ds.cache(),
  validation_data=int_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.6363 - loss: 0.6242 - val_accuracy: 0.8386 - val_loss: 0.3952
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 37s 39ms/step - accuracy: 0.8322 - loss: 0.4104 - val_accuracy: 0.8654 - val_loss: 0.3414
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.8788 - loss: 0.3251 - val_accuracy: 0.8784 - val_loss: 0.3154
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9069 - loss: 0.2717 - val_accuracy: 0.8658 - val_loss: 0.3262
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9158 - loss: 0.2407 - val_accuracy: 0.8794 - val_loss: 0.3339
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.9275 - loss: 0.2160 - val_accuracy: 0.8872 - val_loss: 0.3065
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.9352 - loss: 0.1868 - val_accuracy: 0.8064 - val_loss: 0.6082
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9471 - loss: 0.1626 - 